In [ ]:
!pip install -q tensorflow kaggle kagglehub
# TensorFlow → for deep learning
# kaggle → for authentication (API key)
# kagglehub → for downloading datasets easily




In [ ]:
from google.colab import files
files.upload()
 # Everyone must upload their own kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"suryadeep2105","key":"064b344b7ce82ab6b95a3da7dc1c014d"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# This puts the API key in the correct location
# Required by Kaggle for security reasons

In [ ]:
import kagglehub

# Download 3 different garbage datasets
path1 = kagglehub.dataset_download("vishallazrus/multi-class-garbage-classification-dataset")
path2 = kagglehub.dataset_download("mostafaabla/garbage-classification")
path3 = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

print(path1)
print(path2)
print(path3)
#These datasets are now stored locally (offline)  Kaggle is NOT needed after this step

100%|██████████| 29.2M/29.2M [00:00<00:00, 109MB/s] 

Extracting files...


Using Colab cache for faster access to the 'garbage-classification' dataset.


100%|██████████| 782M/782M [00:03<00:00, 237MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/vishallazrus/multi-class-garbage-classification-dataset/versions/1
/kaggle/input/garbage-classification
/root/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/10


In [ ]:
import os

FINAL_DATASET = "/content/final_dataset"

FINAL_CLASSES = [
    "plastic",
    "metal",
    "paper",
    "glass",
    "battery"
]


# Create one folder per class
for cls in FINAL_CLASSES:
    os.makedirs(os.path.join(FINAL_DATASET, cls), exist_ok=True)

print("Final dataset folders created")


Final dataset folders created


In [ ]:
import shutil

MAP_1 = {  # dataset 1
    "plastic": "plastic",
    "metal": "metal",
    "paper": "paper",
    "glass": "glass",
    "battery": "battery"
}

MAP_2 = {  # dataset 2
    "plastic": "plastic",
    "metal": "metal",
    "paper": "paper",
    "glass": "glass",
    "battery": "battery"
}

MAP_3 = {  # dataset 3
    "plastic": "plastic",
    "metal": "metal",
    "paper": "paper",
    "glass": "glass",
    "battery": "battery"
}

def merge_dataset(src_root, class_map):
    for src_class, final_class in class_map.items():
        src = os.path.join(src_root, src_class)
        dst = os.path.join(FINAL_DATASET, final_class)
        if not os.path.exists(src):
            continue
        for img in os.listdir(src):
            try:
                shutil.copy(os.path.join(src, img), dst)
            except:
                pass

# Merge all datasets
merge_dataset(path1, MAP_1)
merge_dataset(path2, MAP_2)
merge_dataset(path3, MAP_3)

print("All datasets merged successfully")


All datasets merged successfully


In [ ]:
#Load dataset for training
#80% → training
#20% → validation (unseen data)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation improves robustness
datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.3,
    horizontal_flip=True
)

# Training data
train_data = datagen.flow_from_directory(
    FINAL_DATASET,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

# Validation data
val_data = datagen.flow_from_directory(
    FINAL_DATASET,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

print(train_data.class_indices)


Found 7282 images belonging to 5 classes.
Found 1818 images belonging to 5 classes.
{'battery': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4}


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Pretrained MobileNetV2
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)

# Output layer = number of classes
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    verbose=1
)


Epoch 1/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 580s 3s/step - accuracy: 0.7310 - loss: 0.7189 - val_accuracy: 0.8163 - val_loss: 0.5314
Epoch 2/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 592s 3s/step - accuracy: 0.8786 - loss: 0.3447 - val_accuracy: 0.8185 - val_loss: 0.5500
Epoch 3/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 566s 2s/step - accuracy: 0.8908 - loss: 0.2853 - val_accuracy: 0.8333 - val_loss: 0.5261
Epoch 4/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 576s 3s/step - accuracy: 0.9073 - loss: 0.2521 - val_accuracy: 0.8366 - val_loss: 0.5079
Epoch 5/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 560s 2s/step - accuracy: 0.9138 - loss: 0.2397 - val_accuracy: 0.8394 - val_loss: 0.5096


In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


57/57 ━━━━━━━━━━━━━━━━━━━━ 24s 421ms/step - accuracy: 0.8132 - loss: 0.5527
Validation Accuracy: 81.46%


In [ ]:

from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

uploaded = files.upload()

class_names = list(train_data.class_indices.keys())

for img_name in uploaded.keys():
    img = image.load_img(img_name, target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    idx = np.argmax(preds)

    print("Predicted Class:", class_names[idx])
    print(f"Confidence: {preds[idx]*100:.2f}%")

    if class_names[idx] == "battery":
        print("⚠️ Hazardous waste – handle carefully")


Saving plastic3.jpg to plastic3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Predicted Class: plastic
Confidence: 99.68%
